---
# Distance to target

In [1]:
%matplotlib inline

In [2]:
from linescanning import optimal, utils
from pRFline.utils import SubjectsDict
import numpy as np
import subprocess
import multiprocessing as mp
import cortex
from scipy import stats

221026-10:38:49,401 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.8.2


In [3]:
sub = "sub-001"
dc = SubjectsDict()
dc.get_target(sub)

1053

In [4]:
nbr = optimal.Neighbours(
    subject=sub,
    verbose=True
)

Initializing SurfaceCalc
Creating subsurfaces for V1_exvivo.thresh


In [65]:
nbr.lh_surf_normals[dc.get_target(sub)]

array([ 0.51111331,  0.83873185, -0.18786183])

In [5]:
cortex.webshow(nbr.whole_roi_v)

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 62693
Stopping server


<JS: window.viewer>

In [86]:
# beam_surf = np.load("/data1/projects/MicroFunc/Jurjen/projects/VE-pRF/derivatives/freesurfer/sub-001/mri/sub-001_ses-2_task-pRF_run-1_space-fsnative_hemi-LR_run-1_desc-beam_bold.func.npy")

# beam_v = cortex.Vertex(beam_surf, subject=sub, vmin=-0.5, vmax=1)
# target_surf = np.zeros_like(nbr.whole_roi)
# target_surf[dc.get_target(sub)] = 1
# target_v = cortex.Vertex(target_surf, subject=sub, vmin=-0.5, vmax=1)

In [87]:
# cortex.webshow({
#     "V1": nbr.whole_roi_v,
#     "target": target_v,
#     "line": beam_v})

In [88]:
# beam_lh = np.load("/data1/projects/MicroFunc/Jurjen/projects/VE-pRF/derivatives/freesurfer/sub-001/mri/sub-001_ses-2_task-pRF_run-1_space-fsnative_hemi-L_run-1_desc-beam_bold.npy")

# sub_beam = nbr.lh_surf.create_subsurface(vertex_mask=beam_lh.astype(bool))
# sub_vert = np.where(sub_beam.subsurface_vertex_map != stats.mode(sub_beam.subsurface_vertex_map)[0][0])[0]
# sub_vert

In [89]:
# np.where(sub_vert == dc.get_target(sub))[0][0]

In [90]:
# print(nbr.lh_surf_normals[dc.get_target(sub)])
# print(sub_beam.vertex_normals[np.where(sub_vert == dc.get_target(sub))[0][0]])

In [5]:
dist_file = f"/data1/projects/MicroFunc/Jurjen/programs/project_repos/pRFline/data/{sub}/{sub}_hemi-L_desc-V1_distances.json"
nbr.distances_to_target(vert_dict=dist_file, hemi="lh")

Reading distances from /data1/projects/MicroFunc/Jurjen/programs/project_repos/pRFline/data/sub-001/sub-001_hemi-L_desc-V1_distances.json


In [7]:
dist_v = np.zeros((nbr.lh_surf_data[0].shape[0]))

for ii in nbr.lh_subsurf_v:
    if int(ii) != dc.get_target(sub):
        dist_v[ii] = nbr.lh_dist_to_targ[str(ii)]

dist_v = cortex.Vertex(dist_v, subject=sub, vmin=0,vmax=6)
cortex.webshow(dist_v)

Started server on port 36527


KeyboardInterrupt: 

In [104]:
nbr.lh_dist_to_targ

{'43': 12.363253,
 '45': 12.157475,
 '46': 12.046452,
 '48': 11.680391,
 '50': 11.507996,
 '51': 11.095724,
 '52': 11.275881,
 '53': 10.683275,
 '54': 10.219498,
 '55': 10.430448,
 '56': 10.411245,
 '57': 9.352628,
 '58': 9.505925,
 '59': 9.795423,
 '60': 8.491896,
 '61': 8.933895,
 '62': 8.500023,
 '63': 7.910524,
 '108': 11.395926,
 '109': 11.163225,
 '110': 10.656919,
 '111': 10.400421,
 '112': 9.884747,
 '114': 10.917812,
 '115': 10.588696,
 '116': 10.5793,
 '117': 9.810025,
 '118': 9.479335,
 '119': 9.335107,
 '120': 10.816839,
 '121': 9.531812,
 '122': 8.501033,
 '123': 8.115459,
 '124': 9.38052,
 '125': 7.86162,
 '126': 7.319787,
 '127': 8.420398,
 '128': 8.934139,
 '129': 9.313881,
 '130': 8.697947,
 '131': 7.409113,
 '132': 7.215033,
 '133': 6.829686,
 '134': 7.490405,
 '135': 8.029051,
 '136': 6.636211,
 '137': 6.441353,
 '138': 6.356671,
 '139': 7.165466,
 '140': 6.572374,
 '141': 6.660928,
 '142': 7.778534,
 '176': 14.190722,
 '177': 14.950808,
 '178': 15.462484,
 '180': 11

In [59]:
mm2 = nbr.find_distance_range(vmax=2)
print(len(mm2))

9


In [9]:
mm2_4 = nbr.find_distance_range(vmin=2,vmax=4)
print(len(mm2_4))

47


In [10]:
mm4_6 = nbr.find_distance_range(vmin=4,vmax=6)
print(len(mm4_6))

107


In [26]:
nbrs = sorted([dc.get_target(sub)]+mm2+mm2_4+mm4_6)
est_idx = np.where(np.array(nbrs) == dc.get_target(sub))[0][0]
est_idx

68

In [33]:
vert = 204
vert in mm4_6

True

In [35]:
mm2_idx,mm2_4_idx,mm4_6_idx = [],[],[]
for ix,vert in enumerate(nbrs):
    if vert in mm2:
        mm2_idx.append(ix)
    elif vert in mm2_4:
        mm2_4_idx.append(ix)
    elif vert in mm4_6:
        mm4_6_idx.append(ix)

In [41]:
print(f"orig {len(mm2)}\t| indexed {len(mm2_idx)}")
print(f"orig {len(mm2_4)}\t| indexed {len(mm2_4_idx)}")
print(f"orig {len(mm4_6)}\t| indexed {len(mm4_6_idx)}")

orig 15	| indexed 15
orig 47	| indexed 47
orig 107	| indexed 107


In [44]:
def dykstra_distance(start,end):
    cmd = ("mris_pmake", "--subject", str(sub), "--hemi", "lh", "--surface", "fiducial", "--mpmProg", "pathFind", "--mpmArgs", f"vertexStart:{start},vertexEnd:{end}", "--mpmOverlay", "distance")

    # print(cmd)
    L = utils.decode(subprocess.check_output(cmd)).splitlines()
    # print(L)
    dist = float(L[1].split(" ")[-2])
    return (end,dist)

def dykstra_distance2(i,start,end):
    cmd = ("mris_pmake", "--subject", str(sub), "--hemi", "lh", "--surface", "fiducial", "--mpmProg", "pathFind", "--mpmArgs", f"vertexStart:{start},vertexEnd:{end}", "--mpmOverlay", "distance")

    # print(cmd)
    L = utils.decode(subprocess.check_output(cmd)).splitlines()
    # print(L)
    dist = float(L[1].split(" ")[-2])
    return (i,end,dist)    

## `pool.apply()`; works

In [45]:
for ii in dc.dict_data:
    print(ii)

sub-001
sub-002
sub-003
sub-007
sub-008


In [46]:
pool = mp.Pool(5)

results = []
def collect_result(result):
    results.append(result)    
    
# search_verts = nbr.lh_subsurf_v
search_verts = nbr.lh_subsurf_v[:5]

# synchronous; works
results = [pool.apply(dykstra_distance, args=(dc.get_target(sub),vert)) for vert in search_verts]
pool.close()

In [19]:
results

[(43, 12.363253),
 (45, 12.157475),
 (46, 12.046452),
 (48, 11.680391),
 (50, 11.507996)]

## `process()`

In [20]:
from multiprocessing import Process

results = {}
for i,vert in enumerate(search_verts):
    process = Process(target=dykstra_distance2, args=(i,dc.get_target(sub),vert))
    process.start()

['Start->End vertices                                       [ 1053->46 ]', 'Total path cost                                          [ 12.046452 ]']
['Start->End vertices                                       [ 1053->43 ]', 'Total path cost                                          [ 12.363253 ]']
['Start->End vertices                                       [ 1053->48 ]', 'Total path cost                                          [ 11.680391 ]']
['Start->End vertices                                       [ 1053->45 ]', 'Total path cost                                          [ 12.157475 ]']
['Start->End vertices                                       [ 1053->50 ]', 'Total path cost                                          [ 11.507996 ]']


In [16]:
process.__dict__

{'_identity': (10,),
 '_config': {'authkey': b'\x10z\x97{\x9d\xa7\x17\xee\x8b}\xf8Qi\xa0\x1a\xec\x9bHg\xa3Pq\xc6S2\x99\xb7(U\xf7\xae\xe6',
  'semprefix': '/mp'},
 '_parent_pid': 449830,
 '_parent_name': 'MainProcess',
 '_popen': <multiprocessing.popen_fork.Popen at 0x7f5a68e5a250>,
 '_closed': False,
 '_name': 'Process-10',
 '_sentinel': 141}

## `pool.starmap_async()`

In [22]:
search_verts

array([43, 45, 46, 48, 50])

In [48]:
pool = mp.Pool(5)

results = []
# async starmap; doesn't work
results = pool.starmap_async(dykstra_distance2, [(i,dc.get_target(sub),vert) for i,vert in enumerate(nbr.lh_subsurf_v)]).get()
pool.close()
results


mris_pmake
	I'm sorry, but an error condition has occurred.
	While I was scanning user options
	I couldn't find surfaceFile.

	Exiting to system with code 20

mris_pmake
	I'm sorry, but an error condition has occurred.
	While I was scanning user options
	I couldn't find surfaceFile.

	Exiting to system with code 20

mris_pmake
	I'm sorry, but an error condition has occurred.
	While I was scanning user options
	I couldn't find surfaceFile.

	Exiting to system with code 20

mris_pmake
	I'm sorry, but an error condition has occurred.
	While I was scanning user options
	I couldn't find surfaceFile.

	Exiting to system with code 20

mris_pmake
	I'm sorry, but an error condition has occurred.
	While I was scanning user options
	I couldn't find surfaceFile.

	Exiting to system with code 20

mris_pmake
	I'm sorry, but an error condition has occurred.
	While I was scanning user options
	I couldn't find surfaceFile.

	Exiting to system with code 20

mris_pmake
	I'm sorry, but an error condition

CalledProcessError: Command '('mris_pmake', '--subject', 'sub-001', '--hemi', 'lh', '--surface', 'fiducial', '--mpmProg', 'pathFind', '--mpmArgs', 'vertexStart:1053,vertexEnd:115', '--mpmOverlay', 'distance')' returned non-zero exit status 20.

In [25]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]


## `pool.starmap()`

In [27]:
pool = mp.Pool(5)

# sync starmap; doesn't work
results3 = pool.starmap(dykstra_distance, [(dc.get_target(sub),vert) for vert in search_verts])
pool.close()


['Start->End vertices                                       [ 1053->45 ]', 'Total path cost                                          [ 12.157475 ]']
['Start->End vertices                                       [ 1053->45 ]', 'Total path cost                                          [ 12.157475 ]']
['Start->End vertices                                       [ 1053->45 ]', 'Total path cost                                          [ 12.157475 ]']
['Start->End vertices                                       [ 1053->45 ]', 'Total path cost                                          [ 12.157475 ]']
['Start->End vertices                                       [ 1053->45 ]', 'Total path cost                                          [ 12.157475 ]']


## `pool.apply_async()`

In [26]:
pool = mp.Pool(5)
results = []
for i,vert in enumerate(search_verts):
    pool.apply_async(dykstra_distance2, args=(i,dc.get_target(sub),vert), callback=collect_result)

pool.close()
pool.join()

4645504348




['Start->End vertices                                       [ 1053->50 ]', 'Total path cost                                          [ 11.507996 ]']
['Start->End vertices                                       [ 1053->50 ]', 'Total path cost                                          [ 11.507996 ]']
['Start->End vertices                                       [ 1053->50 ]', 'Total path cost                                          [ 11.507996 ]']
['Start->End vertices                                       [ 1053->50 ]', 'Total path cost                                          [ 11.507996 ]']
['Start->End vertices                                       [ 1053->50 ]', 'Total path cost                                          [ 11.507996 ]']


In [100]:
results

[(3, 180, 25.10491),
 (4, 187, 25.10491),
 (1, 178, 25.10491),
 (2, 179, 25.10491),
 (0, 177, 25.10491)]

## `Parallel()`

In [50]:
from joblib import Parallel, delayed
results = Parallel(5,verbose=True)(
    delayed(dykstra_distance)(
        dc.get_target(sub),
        vert
    ) for i,vert in enumerate(search_verts)
)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    4.1s remaining:    6.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    4.4s finished


In [51]:
results

[(43, 12.363253),
 (45, 12.157475),
 (46, 12.046452),
 (48, 11.680391),
 (50, 11.507996)]